# Summarizing metrics from disturbed XS

Leanne Friedrich 11/15/22

## initialization

In [ ]:
%load_ext autoreload
%autoreload 2
LOGGERDEFINED = False

In [ ]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
from py.tools.config import cfg
import py.tools.logs as logs
from plainIm import plainIm
import matplotlib.pyplot as plt
import py.metric_plots as mp
import py.metrics_disturb as me
import regression as rg
import matplotlib.cm as cm
import py.file_handling as fh
import pandas as pd
LOGGERDEFINED = logs.openLog('summary_xs.ipynb', LOGGERDEFINED, level='DEBUG', exportLog=False)

## analyzing data

In [ ]:
me.xsDisturbSummaries(os.path.join(cfg.path.server, 'singleDisturb'), os.path.join(cfg.path.fig, 'singleDisturb'), overwrite=False)

In [ ]:
msy = me.metricDisturbSummary(os.path.join(cfg.path.fig, 'singleDisturb', 'xs+yDisturbSummaries.csv'), diag=True)
print('\n\n')
msz = me.metricDisturbSummary(os.path.join(cfg.path.fig, 'singleDisturb', 'xs+zDisturbSummaries.csv'), diag=True)

In [ ]:
def waterOilPlots(ms:pd.DataFrame, yvar:str, yideal:float, mode:str='ppt', export:bool=False):
    if mode=='ppt':
        figsize = (9,6)
        fs = 18
        markersize=100
    elif mode=='paper':
        figsize=(6,4)
        fs = 10
        markersize=20
    fig,axs = plt.subplots(1,2,figsize=figsize, sharey=True, sharex=True)
    ms.ss.sort_values(by=['sigma', 'sup_val'], inplace=True)
    for i,sigma in enumerate(ms.ss.sigma.unique()):
        mp.scatterSS(ms, ms.ss[ms.ss.sigma==sigma], 'spacing', yvar, 'sup_val'
                 ,fig=fig, ax=axs[i], logx=False, yideal=yideal, xideal=0.875, fontsize=fs, markersize=markersize, legend=False)
    for ax in axs:
        mp.setSquare(ax)
        if ms.ss[yvar].min()<0:
            yy = ms.ss[yvar].min()*0.9
        else:
            yy = ms.ss[yvar].max()*0.9
        ax.text(0.88, yy, 'space filling', color='#555555', fontsize=fs)
        ax.text(1.08, 0.01, 'ideal', color='#555555', fontsize=fs)
    axs[0].set_title('Water/water', fontsize=fs)
    axs[1].set_title('Oil/water', fontsize=fs)
    fig.tight_layout()
    if '+y' in ms.ss.loc[0].printFolder:
        fig.supylabel('In plane')
    else:
        fig.supylabel('Out of plane')
    if export:
        fig.savefig(os.path.join(cfg.path.fig, 'singleDisturb', 'figures', f'horiz_{yvar}.svg'), bbox_inches='tight', dpi=300)

In [ ]:
for ms in [msy, msz]:
    for s in ['delta_aspect', 'delta_h_n', 'delta_w_n', 'delta_yshift', 'delta_xshift', 'delta_xc_n']:
        waterOilPlots(ms, s, 0)

In [ ]:
inspect =ms.ss[ms.ss.bot_w_dxprint>0.1]
inspect

In [ ]:
folder = inspect.iloc[1]['printFolder']
fh.openExplorer(folder)

In [ ]:
me.vertDisturbMeasures(folder, overwrite=True, diag=1, lines=['l0wo', 'l0do', 'l2wo', 'l2do'])

In [ ]:
me.vertDisturbMeasures(folder, overwrite=True, diag=1, lines=['l0w', 'l0d', 'l2w', 'l2d'])

In [ ]:
me.vertDisturbMeasures(folder, overwrite=True, diag=1)
me.vertDisturbSummary(folder, overwrite=True);